# Marine Carbonate System Visualizer - TA & DIC

## Interactive Bjerrum Plot for Marine Chemistry

<div style="font-size: 22px; line-height: 1.6;">

<div style="text-align: center; margin: 30px 0;">
    <img src="figures/carbonate_system.svg" width="800" alt="Marine Carbonate System Diagram" style="border-radius: 10px; box-shadow: 0 4px 8px rgba(0,0,0,0.1); max-width: 100%;">
</div>

<div style="text-align: center; font-size: 18px; color: #00204C; margin: 20px 0;">
    <strong>Comprehensive visualization using Total Alkalinity and DIC variations</strong>
</div>

</div>

This tool visualizes the marine carbonate system using **Bjerrum plots** with **Total Alkalinity (TA)** and **Dissolved Inorganic Carbon (DIC)** as input parameters.

**Features:**
- Interactive sliders for Total Alkalinity and DIC
- 4-panel visualization showing carbonate speciation
- Real-time calculations using PyCO2SYS
- Educational tool for marine chemistry courses

**Instructions:**
1. Adjust the sliders below to change Total Alkalinity and DIC values
2. Observe how the carbonate system responds in real-time
3. Use the Reset button to return to default values

## What does this tool show?

**Input parameters:**
- **Total alkalinity** (TA) in μmol/kg
- **Dissolved Inorganic Carbon** (DIC) in μmol/kg

**Fixed conditions:**
- **Salinity** = 35 PSU
- **Temperature** = 25°C

**Visualizations:**
1. **Bjerrum Plot** - Species fractions vs pH
2. **Current Composition** - Pie chart of species
3. **Saturation State** - Aragonite saturation
4. **System Analysis** - Detailed results

**Educational Value:**
- Understand how TA and DIC define the carbonate system
- Explore effects of mixing different water masses
- Analyze biological effects on carbonate chemistry
- Study calcification potential under different conditions

In [1]:
# Load necessary libraries
import PyCO2SYS as pyco2
import matplotlib.pyplot as plt
import numpy as np
from ipywidgets import FloatSlider, VBox, interactive_output, Button, Layout, HTML
from IPython.display import display

In [2]:
# 🔧 Marine Carbonate System Configuration
# These are the standard seawater conditions for all calculations

CONFIG = {
    'salinity': 35,         # Typical ocean salinity (PSU)
    'temperature': 25,      # Surface temperature (°C)
    'pressure': 0,          # Sea surface pressure (dbar)
    'opt_pH_scale': 1,      # Total pH scale (recommended for seawater)
    'opt_k_carbonic': 10    # Waters et al. (2014) constants (most accurate)
}

def set_config(**kwargs):
    """Modify calculation conditions for different environments
    
    Examples for students:
    set_config(temperature=15)                # Cold water conditions
    set_config(salinity=30, temperature=10)   # Estuarine conditions
    set_config(opt_k_carbonic=8)             # Use Lueker et al. (2000) constants
    """
    for key, value in kwargs.items():
        if key in CONFIG:
            old_value = CONFIG[key]
            CONFIG[key] = value
            print(f"✓ Changed {key}: {old_value} → {value}")
        else:
            print(f"⚠ Unknown parameter: {key}")
            print(f"📋 Available parameters: {list(CONFIG.keys())}")
    
    print(f"\n🌊 Current seawater conditions: {CONFIG}")

# Educational information about constants
print("🔬 Marine Carbonate System Configuration Ready!")
print("📚 Carbonic acid constants options:")
print("   8  = Lueker et al. (2000) - Widely used")
print("   10 = Waters et al. (2014) - Current best practice ⭐")
print("   14 = Schockman & Byrne (2021) - Latest research")
print("\n💡 Students: Try set_config(temperature=15) to see cold water effects!")

🔬 Marine Carbonate System Configuration Ready!
📚 Carbonic acid constants options:
   8  = Lueker et al. (2000) - Widely used
   10 = Waters et al. (2014) - Current best practice ⭐
   14 = Schockman & Byrne (2021) - Latest research

💡 Students: Try set_config(temperature=15) to see cold water effects!


In [3]:
# ⚗️ Marine Carbonate Chemistry Calculator (TA-DIC Method)

def get_pyco2_params(alkalinity, DIC):
    """Prepare parameters for TA-DIC calculations
    
    This function uses Total Alkalinity and DIC
    to calculate the complete carbonate system.
    """
    return {
        'par1': alkalinity,         # Input 1: Total Alkalinity (μmol/kg)
        'par2': DIC,               # Input 2: Dissolved Inorganic Carbon (μmol/kg)
        'par1_type': 1,            # Parameter type 1 = Total Alkalinity
        'par2_type': 2,            # Parameter type 2 = DIC
        'salinity': CONFIG['salinity'],
        'temperature': CONFIG['temperature'],
        'pressure': CONFIG['pressure'],
        'opt_pH_scale': CONFIG['opt_pH_scale'],
        'opt_k_carbonic': CONFIG['opt_k_carbonic']
    }

def compute_carbonate_system(alkalinity, DIC):
    """Calculate complete carbonate system from TA and DIC
    
    Students: This function shows how TA and DIC define the system:
    - TA controls buffering capacity
    - DIC controls total carbon available
    - Together they determine pH, pCO2, and saturation states
    """
    try:
        # Run the calculation using PyCO2SYS
        results = pyco2.sys(**get_pyco2_params(alkalinity, DIC))
        
        # Extract and organize results for plotting
        return {
            "pH_total": float(results["pH_total"]),
            "pCO2": float(results["pCO2"]), 
            "bicarbonate": float(results["bicarbonate"]),
            "carbonate": float(results["carbonate"]),
            "DIC": float(results["dic"]),  # PyCO2SYS uses lowercase 'dic'
            "omega_aragonite": float(results["saturation_aragonite"]),
            "alkalinity": float(results["alkalinity"]),
            **CONFIG
        }
        
    except KeyError as e:
        print(f"❌ Data Key Error: {e}")
        print("📋 This usually means PyCO2SYS result keys changed")
        print("💡 Try updating PyCO2SYS: pip install --upgrade PyCO2SYS")
        raise
        
    except Exception as e:
        print(f"❌ Calculation Error: {e}")
        print(f"📋 Check your inputs: TA={alkalinity}, DIC={DIC}")
        print("💡 Typical values: TA=2300 μmol/kg, DIC=2000 μmol/kg")
        raise

def get_constants_from_current_system(data):
    """Extract equilibrium constants for Bjerrum plot
    
    Students: This calculates K1 and K2 from the current system state
    to draw the species distribution curves correctly.
    """
    try:
        pH = data['pH_total']
        pCO2 = data['pCO2'] 
        HCO3 = data['bicarbonate']
        CO3 = data['carbonate']
        
        # Calculate [H+] and [CO2*] concentrations
        H = 10**(-pH)
        CO2_star = pCO2 * 0.034  # Henry's law constant for CO2
        
        # Calculate equilibrium constants from concentrations
        K1 = (H * HCO3 / CO2_star) if CO2_star > 0 else 1e-6
        K2 = (H * CO3 / HCO3) if HCO3 > 0 else 1e-9
        
        return K1, K2
        
    except Exception as e:
        print(f"⚠ Constants calculation issue: {e}")
        return 1e-6, 1e-9  # Safe fallback values

print("⚗️ TA-DIC calculation functions ready!")
print("🌊 This tool shows how TA and DIC define the carbonate system")
print("🎯 Perfect for studying complete carbonate system relationships")

⚗️ TA-DIC calculation functions ready!
🌊 This tool shows how TA and DIC define the carbonate system
🎯 Perfect for studying complete carbonate system relationships


---

## 🚀 How to Use This Tool

### Step 1: Configure PyCO2SYS (Above)
- **Default settings** work for typical seawater conditions
- **To change**: Use `set_config(parameter=value)` in the configuration cell
- **Common changes**: 
  - `set_config(opt_k_carbonic=8)` - Use Lueker et al. (2000) constants
  - `set_config(temperature=15, salinity=30)` - Different conditions

### Step 2: Run the Interactive Interface (Below)
- Move the **sliders** to explore different TA and DIC values
- **Plots update automatically** showing:
  1. **Bjerrum Plot** - Species fractions vs pH
  2. **Composition** - Current system breakdown  
  3. **Saturation** - Aragonite saturation state
  4. **Analysis** - Detailed results with your chosen constants

### Step 3: Analyze Results
- **Red line** in Bjerrum plot shows current pH
- **TA/DIC variations** show complete system effects
- **All calculations** use the same consistent parameters

### Educational Applications:
- **Complete System**: TA and DIC uniquely define everything
- **Water Mass Mixing**: See effects of different TA/DIC ratios
- **Biological Effects**: Understand impact on calcification
- **Buffering Capacity**: How TA controls pH stability

---

In [4]:
# 📊 Plotting Functions - Bjerrum Diagram and Analysis

def create_bjerrum_plot(data):
    """Create 4-panel Bjerrum plot with system analysis for TA-DIC"""
    try:
        plt.close('all')
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(14, 10))
        fig.suptitle('Marine Carbonate System Analysis (TA-DIC)', fontsize=16, fontweight='bold')
        
        colors = ['#443983', '#31688e', '#35b779', '#fde725']
        pH_range = np.linspace(4, 11, 40)
        
        # Get constants from current system state
        K1, K2 = get_constants_from_current_system(data)
        
        # Calculate speciation using analytical expressions
        H_values = 10**(-pH_range)
        denom = H_values**2 + K1*H_values + K1*K2
        
        # Ensure denominator is never zero
        denom = np.where(denom == 0, 1e-15, denom)
        
        alpha0 = H_values**2 / denom      # CO2
        alpha1 = K1*H_values / denom      # HCO3-
        alpha2 = K1*K2 / denom            # CO3-2
        
        # Plot 1: Bjerrum diagram
        ax1.plot(pH_range, alpha0, color=colors[0], linewidth=3, label='CO2')
        ax1.plot(pH_range, alpha1, color=colors[1], linewidth=3, label='HCO3-')
        ax1.plot(pH_range, alpha2, color=colors[2], linewidth=3, label='CO3-2')
        
        current_pH = data['pH_total']
        ax1.axvline(x=current_pH, color='red', linestyle='--', linewidth=2)
        ax1.text(current_pH + 0.1, 0.8, f'pH: {current_pH:.2f}', fontsize=10, color='red')
        ax1.set_xlabel('pH'), ax1.set_ylabel('Fraction of DIC')
        ax1.set_title('Bjerrum Plot'), ax1.legend(), ax1.grid(True, alpha=0.3)
        ax1.set_xlim(4, 11), ax1.set_ylim(0, 1)
        
        # Plot 2: Composition pie chart
        CO2_conc = data["pCO2"] * 0.034  # Henry's law
        species = [CO2_conc, data["bicarbonate"], data["carbonate"]]
        species_labels = ['CO2*', 'HCO3-', 'CO3-2']
        
        ax2.pie(species, labels=species_labels, autopct='%1.1f%%', colors=colors[:3])
        ax2.set_title(f'Composition at pH {current_pH:.2f}')
        
        # Plot 3: Saturation state
        omega = data["omega_aragonite"]
        color_omega = colors[3] if omega >= 1 else colors[1]
        ax3.bar(['Aragonite'], [omega], color=color_omega)
        ax3.axhline(y=1, color='black', linestyle='--', alpha=0.7)
        ax3.set_ylabel('Omega (Ω)'), ax3.set_title('Saturation State')
        
        # Add saturation status
        status = "Supersaturated" if omega >= 1 else "Undersaturated"
        ax3.text(0, omega*0.7, f'Ω = {omega:.2f}\n{status}', ha='center', fontsize=11)
        
        # Plot 4: System information
        ax4.axis('off')
        
        # Calculate display values
        TA_calc = data.get('alkalinity', data['bicarbonate'] + 2*data['carbonate'])
        DIC_calc = data['DIC']
        
        # Use text without problematic emojis for matplotlib compatibility
        info_text = f"""INPUTS:
TA = {TA_calc:.0f} umol/kg
DIC = {DIC_calc:.0f} umol/kg

RESULTS:
pH = {current_pH:.2f}
pCO2 = {data['pCO2']:.0f} uatm
[HCO3-] = {data['bicarbonate']:.0f} umol/kg
[CO3-2] = {data['carbonate']:.0f} umol/kg
Omega_arag = {omega:.2f}

CONSTANTS:
K1 = {K1:.2e}
K2 = {K2:.2e}

CONDITIONS:
S = {data['salinity']} PSU
T = {data['temperature']}°C
pH scale = {data['opt_pH_scale']}
K constants = {data['opt_k_carbonic']}

CONTEXT:
Typical TA: 2200-2400 umol/kg
Typical DIC: 1900-2200 umol/kg
Ocean pH: 7.8-8.2"""
        
        ax4.text(0.05, 0.95, info_text, transform=ax4.transAxes, fontsize=9,
                 verticalalignment='top', fontfamily='monospace')
        
        plt.tight_layout()
        plt.show()
        return fig
        
    except Exception as e:
        print(f"Error creating plot: {e}")
        print(f"   Data keys: {list(data.keys()) if isinstance(data, dict) else 'Not a dict'}")
        raise

print("📊 Robust plotting functions ready for TA-DIC analysis!")
print("🛡️ Enhanced error handling and data validation")

📊 Robust plotting functions ready for TA-DIC analysis!
🛡️ Enhanced error handling and data validation


In [ ]:
# 🎛️ Interactive Interface - TA and DIC Sliders

# Create sliders
sliders = {
    'alk': FloatSlider(value=2300, min=1800, max=3500, step=50,
                      description="Alkalinity (μmol/kg):", 
                      style={'description_width': 'initial'}, layout=Layout(width='600px')),
    'dic': FloatSlider(value=2020, min=1600, max=3500, step=50,
                      description="DIC (μmol/kg):",
                      style={'description_width': 'initial'}, layout=Layout(width='600px'))
}

# Update function (no duplicates)
def update_plots(alkalinity, DIC):
    """Update plot when sliders change"""
    data = compute_carbonate_system(alkalinity, DIC)
    create_bjerrum_plot(data)

# Reset function
def reset_values(b):
    """Reset sliders to default values"""
    sliders['alk'].value, sliders['dic'].value = 2300, 2020

# Preset buttons for common scenarios
def set_typical_ocean(b):
    """Set to typical open ocean conditions"""
    sliders['alk'].value = 2300
    sliders['dic'].value = 2020
    
def set_surface_water(b):
    """Set to surface water conditions (lower DIC)"""
    sliders['alk'].value = 2300
    sliders['dic'].value = 1900
    
def set_deep_water(b):
    """Set to deep water conditions (higher DIC)"""
    sliders['alk'].value = 2350
    sliders['dic'].value = 2200

# Create buttons
reset_button = Button(description="🔄 Reset Values", button_style='info')
ocean_button = Button(description="🌊 Typical Ocean", button_style='success')
surface_button = Button(description="☀️ Surface Water", button_style='warning')
deep_button = Button(description="🏔️ Deep Water", button_style='danger')

# Connect buttons to functions
reset_button.on_click(reset_values)
ocean_button.on_click(set_typical_ocean)
surface_button.on_click(set_surface_water)
deep_button.on_click(set_deep_water)

# Create button layout
button_box = VBox([
    HTML("<h4>🌊 Quick TA-DIC Scenarios:</h4>"),
    VBox([ocean_button, surface_button, deep_button, reset_button], 
         layout=Layout(width='100%'))
])

# Display complete interface
display(VBox([
    HTML("<h3>🌊 Marine Carbonate System Explorer - TA & DIC</h3>"),
    HTML("<p><em>📊 Explore complete carbonate system using Total Alkalinity and DIC</em></p>"),

    sliders['alk'], 
    sliders['dic'],
    button_box,
    interactive_output(update_plots, {'alkalinity': sliders['alk'], 'DIC': sliders['dic']})
]))

print("🎯 Interactive TA-DIC carbonate system interface ready!")
print("📋 All calculations use PyCO2SYS with your chosen constants")
print("💡 Use the scenario buttons to explore different water masses")
print("🌊 Typical ocean: TA≈2300, DIC≈2020 μmol/kg")
print("☀️ Surface waters: Lower DIC due to biological uptake")
print("🏔️ Deep waters: Higher DIC from respiration")

🎯 Interactive TA-DIC carbonate system interface ready!
📋 All calculations use PyCO2SYS with your chosen constants
💡 Use the scenario buttons to explore different water masses
🌊 Typical ocean: TA≈2300, DIC≈2020 μmol/kg
☀️ Surface waters: Lower DIC due to biological uptake
🏔️ Deep waters: Higher DIC from respiration


**Author:** Cardoso-Mohedano JG  
**Institution:** Instituto de Ciencias del Mar y Limnologia, UNAM, Estacion El Carmen  
**License:** [CC BY-NC 4.0](https://creativecommons.org/licenses/by-nc/4.0/)  
**ORCID:** [0000-0002-2918-972X](https://orcid.org/0000-0002-2918-972X)

---
This interactive tool allows exploration of the marine carbonate system using [PyCO2SYS](https://pyco2sys.readthedocs.io/en/latest/).  
Developed with support from [Claude AI](https://claude.ai) by Anthropic and [OpenAI ChatGPT](https://openai.com/chatgpt) and educational Python tools.

### TA-DIC Educational Notes:

**Why TA and DIC?**
- **Complete Definition**: TA and DIC uniquely define the entire carbonate system
- **Conservative Parameters**: Not affected by temperature/pressure changes during water mass transport
- **Oceanographic Relevance**: Used to track water masses and biological processes

**Key Learning Objectives:**
1. **System Definition**: How two parameters determine all others
2. **Water Mass Analysis**: Different TA/DIC signatures of ocean regions
3. **Biological Effects**: Impact of photosynthesis and respiration
4. **Mixing Processes**: Conservative vs non-conservative behavior